# Neptune ML

In this notebook, we will walk through the process of uploading our processed data to an Amazon Neptune database, querying the database with Gremlin, and using Neptune ML to train and deploy a machine learning-powered link prediction model.

As a first step, read through the following documentation.  When you're ready to proceed, click the "Launch Stack" button for your desired AWS region to launch the CloudFormation stack in your account.  This will take care of creating the Neptune DB instance as well as the necessary infrastructure for Neptune ML.

https://docs.aws.amazon.com/neptune/latest/userguide/machine-learning.html

Once the stack has been fully deployed, launch this notebook on the SageMaker Notebook instance that has been created by the stack.  Then you may run the following cells to import the included Neptune ML utilities we will use later on.

In [1]:
import sys
sys.path.insert(0,'/home/ec2-user/SageMaker/Neptune/04-Machine-Learning')

In [2]:
import neptune_ml_utils as neptune_ml
neptune_ml.check_ml_enabled()

This Neptune cluster is configured to use Neptune ML


In [3]:
s3_bucket_uri = "s3://hcls-kg-workshop"

## Reset cluster (optional)

Before we begin, we may clear out existing data from the Neptune DB instance if desired.

In [4]:
import requests
url = 'https://' + neptune_ml.get_host() + ":8182/system"

### Initiate reset

In [5]:
init_headers = {'Content-Type': 'application/json'}
init_payload = '{ "action" : "initiateDatabaseReset" }'
init_result = requests.post(url, data=init_payload, headers=init_headers)
print(init_result.json())

{'status': '200 OK', 'payload': {'token': '9ec0dac5-e8a6-900e-fd07-12e66d7f1678'}}


### Execute reset

In [6]:
exec_headers = {'Content-Type': 'application/x-www-form-urlencoded'}
exec_payload = 'action=performDatabaseReset&token=' + init_result.json()['payload']['token']
exec_result = requests.post(url, data=exec_payload, headers=exec_headers)
print(exec_result.json())

{'status': '200 OK'}


## Load data from S3 into cluster

We set the value of `s3_prefix` in the next cell to the S3 path to the CSVs processed for Gremlin in the prior notebook.  Then run the subsequent cell with the `%load` line magic to upload all of the nodes and edges we have created to the Neptune DB.

In [9]:
import os

# s3_prefix = "gremlin_data/2021_08_04_13_17_13/"
# s3_prefix = "stdized-data-new/comprehend_results/gremlin_data_repeat/"
s3_prefix = "stdized-data-new/comprehend_results/gremlin_data_from_pubmed_reduced100_node_v1_2/"
s3_location = os.path.join(s3_bucket_uri, s3_prefix)

In [10]:
%load -s {s3_location} -f csv -p OVERSUBSCRIBE --run

Button(description='Submit', style=ButtonStyle())

Output()

## Query the Neptune DB

Once the Neptune upload has completed, we can use the `%%gremlin` cell magic to write queries to the database using the Gremlin language.

### Nodes: count by label

In [66]:
%%gremlin
g.V().groupCount().by(label).unfold()

### Edges: count by label

In [33]:
%%gremlin
g.E().groupCount().by(label).unfold()

### Visualize

In [76]:
%%gremlin -p v,ine,outv,oute,inv,oute,inv
g.V().hasLabel('SYSTEM_ORGAN_SITE').limit(1).
inE().outV().outE().inV().outE().inV().path().by(valueMap(true))

## Export the relevant data and model configuration

In this section, we leverage the Neptune ML utilities to export data from the Neptune DB instance back to S3 for machine learning.  The `export_params` dictionary specifies the relevant parameters for the export job depending on the problem we wish to solve.  In this case, we focus on the `DX_NAME` and `SYSTEM_ORGAN_SITE` nodes and the `SYSTEM_ORGAN_SITE` edges that connect them, with the goal of modeling those connections with machine learning.

In [13]:
# DB Cluster
neptune_ml.get_host()

'neptunedbcluster-hjhnyudeb2ac.cluster-cwevnqts1jz0.us-east-2.neptune.amazonaws.com'

In [14]:
# Export service
neptune_ml.get_export_service_host()

'pdsqjuffzk.execute-api.us-east-2.amazonaws.com/v1'

In [ ]:
# Define export params
export_params = {
    "params": {
        "endpoint": neptune_ml.get_host(),
        "profile" : "neptune_ml",
        "cloneCluster": False,
        "filter" : {
          "nodes" : [
            {
              "label" : "DX_NAME",
              "properties" : ["Name"]
            },
            {
              "label" : "SYSTEM_ORGAN_SITE",
              "properties" : ["Name"]
            }
          ],
          "edges" : [
            {
              "label" : "SYSTEM_ORGAN_SITE",
              "properties" : ["RelationshipScore"]
            }
          ]
        },
        "nodeLabels" : ["DX_NAME", "SYSTEM_ORGAN_SITE"],
        "edgeLabels" : ["SYSTEM_ORGAN_SITE"],
    }, 
    "outputS3Path": f'{s3_bucket_uri}/neptune-export',
    "additionalParams": {
        "neptune_ml": {
            "version": "v2.0",
            "targets": [
                {
                    "edge": ["SYSTEM_ORGAN_SITE", "SYSTEM_ORGAN_SITE", "DX_NAME"],
                    "type" : "link_prediction",
                    "split_rate": [0.9, 0.1, 0.0]
                }
            ],
            "features": [
                {
                    "node": "SYSTEM_ORGAN_SITE",
                    "property": "Name",
                    "type": "auto"
#                     "type": "text_word2vec",
#                     "language": "en_core_web_lg"
                },
                {
                    "node": "DX_NAME",
                    "property": "Name",
                    "type": "auto"
#                     "type": "text_word2vec",
#                     "language": "en_core_web_lg"
                },
                {
                    "edge": ["SYSTEM_ORGAN_SITE", "SYSTEM_ORGAN_SITE", "DX_NAME"],
                    "property": "RelationshipScore",
                    "type": "auto"
#                     "type": "numerical",
#                     "norm": "none"
                },
            ]
        }
    },
    "jobSize": "medium"
}

Use cell magic to start the export.

In [ ]:
%%neptune_ml export start --export-url {neptune_ml.get_export_service_host()} --export-iam --wait --store-to export_results 
${export_params}

In [ ]:
# Check output S3 location
export_results['outputS3Uri']

## Data processing

In this section, we use the Neptune ML utilities to run a data processing job in the exported data given the specified `processing_params`.

In [ ]:
# The training_job_name can be set to a unique value below, otherwise one will be auto generated
training_job_name=neptune_ml.get_training_job_name('link-prediction')

processing_params = f"""
--config-file-name training-data-configuration.json
--job-id {training_job_name}
--s3-input-uri {export_results['outputS3Uri']}
--s3-processed-uri {str(s3_bucket_uri)}/preloading"""

Use the line magic to start the data processing.

In [ ]:
%neptune_ml dataprocessing start --wait --store-to processing_results {processing_params}

## Model training

Now that we've exported and processed the relevant data, we are ready to train our link prediction model.  Here we can specify the `instance_type` we would like to use to train our model, as well as the destination for the trained model object in S3.

In [ ]:
training_params=f"""
--job-id {training_job_name}
--data-processing-id {training_job_name}
--instance-type ml.p3.2xlarge
--s3-output-uri {str(s3_bucket_uri)}/training"""

Use line magic to train the model.

In [ ]:
%neptune_ml training start --wait --store-to training_results {training_params}

## Endpoint creation

Finally, we can use Neptune ML to launch an endpoint that serves our trained model.  Importantly, this endpoint can be accessed from within Gremlin to seamlessly leverage model inference as part of a graph query.

In [ ]:
endpoint_params=f"""
--id {training_job_name}
--model-training-job-id {training_job_name}"""

Use line magic to create the endpoint.

In [ ]:
%neptune_ml endpoint create --wait --store-to endpoint_results {endpoint_params}

In [ ]:
# Get the endpoint name
endpoint = endpoint_results['endpoint']['name']
endpoint

## Using the Endpoint to query the graph

In this final section, we leverage our trained model endpoint to make link predictions using Gremlin.  In the first example, we begin with a specific `DX_NAME` node and use our ML model to predict the most probable connections to `SYSTEM_ORGAN_SITE` nodes.  In the second example we do the reverse, starting with a specific `SYSTEM_ORGAN_SITE` node and predicting the most probable connections to `DX_NAME` nodes.  In both examples, we compare the true connections that exist in the graph to the connections predicted by the model.

### Predict potential organ systems for a diagnosis

In [ ]:
%%gremlin
g.V().hasLabel('DX_NAME').limit(10).valueMap()

In [ ]:
# Set diagnosis name to analyze with ML
diagnosis_name = 'infection'

In [ ]:
%%gremlin
g.V().hasLabel('DX_NAME').
    has('Name', '${diagnosis_name}').
    out('SYSTEM_ORGAN_SITE').
    hasLabel('SYSTEM_ORGAN_SITE').valueMap()

In [ ]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint}").
    with("Neptune#ml.limit", 10).
    V().hasLabel('DX_NAME').
    has('Name', '${diagnosis_name}').
    out('SYSTEM_ORGAN_SITE').
    with("Neptune#ml.prediction").
    hasLabel('SYSTEM_ORGAN_SITE').
    valueMap()

### Predict potential diagnoses for an organ system

In [ ]:
%%gremlin
g.V().hasLabel('SYSTEM_ORGAN_SITE').limit(10).valueMap()

In [ ]:
# Set organ system to analyze with ML
organ_system_name = 'brain'

In [ ]:
%%gremlin
g.V().hasLabel('SYSTEM_ORGAN_SITE').
    has('Name', '${organ_system_name}').
    in('SYSTEM_ORGAN_SITE').
    hasLabel('DX_NAME').valueMap()

In [ ]:
%%gremlin
g.with("Neptune#ml.endpoint","${endpoint}").
    with("Neptune#ml.limit", 10).
    V().hasLabel('SYSTEM_ORGAN_SITE').
    has('Name', '${organ_system_name}').
    in('SYSTEM_ORGAN_SITE').
    with("Neptune#ml.prediction").
    hasLabel('DX_NAME').
    valueMap()

## Delete endpoint (optional)

We can use the Neptune ML utilities to delete the SageMaker endpoint when we're done if we'd like.

In [ ]:
neptune_ml.delete_endpoint(training_job_name)